# Sissejuhatus Pandase ja Matplotlibi kasutamisse

## Mis on Pandas?
Pandas on populaarne Python'i teek, mis võimaldab mugavalt töödelda tabelandmeid. Seda kasutatakse andmeteaduses, analüüsis ja visualiseerimises. Pandas teeb andmete puhastamise, filtreerimise, sorteerimise ja analüüsi lihtsaks.

Pandas ja Matplotlib on olulised Python'i teegid andmete töötlemiseks ja visualiseerimiseks. Andmeteaduses on tähtis osata andmeid tõhusalt hallata ja tulemusi selgelt esitada.

## Andmeteaduse töövoog
Tüüpiline töövoog: andmete laadimine → puhastamine/ettevalmistus → analüüs → visualiseerimine → tulemuste esitamine. Pandast kasutatakse andmete haldamiseks ja analüüsiks, Matplotlibi visualiseerimiseks.

**Hea praktika:**
- Ära muuda originaalset DataFrame'i, vaid loo iga olulisema sammu jaoks uus muutuja (nt df_clean, df_sorted). Nii on töövoog jälgitav ja vigu lihtsam leida.
- Kui teed järjestikku mitu operatsiooni, kasuta Pandase .pipe() meetodit, mis võimaldab funktsioone järjestada ja koodi loetavust parandada.

Näide [pipe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html) kasutamisest:

df_clean = (
    df.dropna()
      .pipe(lambda d: d[d['age'] > 18])
      .sort_values('income', ascending=False)
)

## Pandase põhikontseptsioonid
Pandasel on kaks põhilist andmestruktuuri: **Series** (1-mõõtmeline) ja **DataFrame** (2-mõõtmeline).
- DataFrame on nagu arvutustabel: iga veerg on Series ja kõigil veergudel on sama indeks.

In [ ]:
import pandas as pd

# Series loomine
s = pd.Series([1, 2, 3, 4])
print(s)

# Näidisandmestiku loomine DataFrame'ina
df = pd.DataFrame({
    'name': ['Anna', 'Jaan', 'Mari', 'Peeter', 'Liis'],
    'age': [22, 35, 19, 42, 28],
    'city': ['Tallinn', 'Tartu', 'Tallinn', 'Pärnu', 'Tartu'],
    'income': [1200, 2100, 950, 1800, 1600]
})
print(df)

## Andmetüübid Pandases
Pandas DataFrame'i iga veerg omab kindlat andmetüüpi, mis määrab, kuidas andmeid töödeldakse ja milliseid operatsioone saab teha. Olulised tüübid on:
- **object**: Tekst (stringid), segatüüpi väärtused. Näide: ['Tallinn', 'Tartu']
- **int**: Täisarvud. Näide: [1, 2, 3]
- **float**: Ujukomaarvud (komaga arvud). Näide: [3.14, 2.0]
- **bool**: Tõeväärtused (True/False). Näide: [True, False, True]
- **datetime64**: Kuupäevad ja kellaajad. Näide: ['2023-01-01', '2024-06-01']

Täisarvude puhul on Pandases mitmeid alamliike, mis määravad, kui palju mälu veerg kasutab:
- **Int16**: 16-bitine täisarv (-32768 kuni 32767). Sobib väikeste arvude jaoks, säästab mälu. Lubab puuduvaid väärtusi (pandas.NA).
- **Int32**: 32-bitine täisarv (-2 147 483 648 kuni 2 147 483 647). Kasutatakse suuremate arvude jaoks. Lubab puuduvaid väärtusi (pandas.NA).

Erinevalt NumPy int tüüpidest lubavad Pandase Int16 ja Int32 tüübid puuduvaid väärtusi, mis on kasulik näiteks ankeetandmete või mittetäielike andmestike puhul.
Kui tead, et veerus on ainult väikeseid arve, kasuta Int16, et vähendada mälukasutust. Suuremate arvude jaoks kasuta Int32 või Int64.

### Näited erinevatest andmetüüpidest Pandases:

In [ ]:
import pandas as pd

df_types = pd.DataFrame({
    'nimi': ['Anna', 'Jaan'],  # object
    'vanus': pd.Series([22, pd.NA], dtype='Int16'),  # Int16 lubab puuduvaid väärtusi
    'palk': [1200.5, 2100.0],  # float
    'aktiivne': [True, False],  # bool
    'liitumise_kuup': pd.to_datetime(['2023-01-01', '2024-06-01'])  # datetime64
})
print(df_types.dtypes)
print(df_types)

# Tüübi muutmine Int32 peale (ka Int32 lubab puuduvaid väärtusi)
df_types['vanus'] = df_types['vanus'].astype('Int32')
print(df_types.dtypes)

Tüüpide kontroll ja muutmine on oluline, et vältida vigu ja optimeerida mälukasutust. Näiteks Int16 sobib väikeste arvude jaoks, Int32 suuremate jaoks. Kuupäevadega töötamiseks kasuta datetime64 tüüpi. Puuduvate väärtuste jaoks kasuta Pandase Int-tüüpe, mitte NumPy int-tüüpe.

## Indekseerimine ja indeksi muutmine
Igal DataFrame'il on indeks, mis määrab ridade järjestuse ja identiteedi. Vaikimisi on indeks numbriline (0, 1, 2, ...), kuid tihti on kasulik kasutada tähenduslikku indeksit, näiteks nime, kuupäeva või unikaalset koodi.

**Miks määrata eraldi indeks?**
- Kiirem ja mugavam andmete otsimine (nt df.loc['Anna'])
- Loogilisem andmete struktuur, eriti kui ridadel on unikaalne tunnus
- Grupitöötlus ja joinimine teiste andmestikega on lihtsam
- Ajalooliste andmete puhul saab kasutada kuupäeva indeksina

Kui ekspordid andmed Excelisse või Parquet-faili ja kasutad index=False, siis indeksit ei salvestata. Selle vältimiseks tee indeksist tulp tagasi.

In [ ]:
# Indeksi määramine veeru põhjal
df_indexed = df.set_index('name')
print(df_indexed)

# Andmete otsimine indeksi abil
print(df_indexed.loc['Anna'])

# Indeksist tulba tegemine (enne eksporti)
df_reset = df_indexed.reset_index()
print(df_reset)

## Andmete sorteerimine
Sorteerimine aitab leida suurimaid/väiksemaid väärtusi või järjestada andmeid loogiliselt.

### Sorteerimine ühe või mitme tulba järgi
Saad sorteerida ühe või mitme tulba järgi, määrates tulbanimed listina. Näiteks vanuse ja sissetuleku järgi:

In [ ]:
# Sorteeri vanuse järgi kahanevalt
df_sorted = df.sort_values('age', ascending=False)
print(df_sorted)

# Sorteeri vanuse ja sissetuleku järgi
df_multi_sorted = df.sort_values(['age', 'income'], ascending=[True, False])
print(df_multi_sorted)

### Sorteerimine custom funktsiooniga
Alates Pandas 1.1 saab kasutada sort_values(key=...), kus key on funktsioon (vaata ka [lambda funktsioone](https://docs.python.org/3/reference/expressions.html#lambda)), mis rakendatakse sorteeritavale veerule. Näiteks sorteerida nime pikkuse järgi.

In [ ]:
# Sorteeri nime pikkuse järgi (custom funktsiooniga, Pandas >=1.1)
df_custom_sorted = df.sort_values('name', key=lambda x: x.str.len())
print(df_custom_sorted)

## Uute veergude lisamine ja arvutamine
Sageli on vaja arvutada uusi veerge olemasolevate põhjal, näiteks arvutada netosissetulek, määrata vanusekategooria või luua bool-tunnus (True/False). Uued veerud aitavad andmeid paremini analüüsida ja visualiseerida.

Uue veeru saab lisada lihtsalt, omistades DataFrame'ile uue tulbanime:
- Arvutatud veerg: df['income_tax'] = df['income'] * 0.24
- Kategooria: df['age_group'] = pd.cut(df['age'], bins=[0,18,65,100], labels=['alaealine','täiskasvanu','eakas'])
- Bool-tunnus: df['is_adult'] = df['age'] >= 18

## Tabelite liitmine (joinimine)
Tabelite liitmine (merge/join) võimaldab ühendada andmeid erinevatest allikatest. Joinimiseks peab olema:
- Ühine veerg (või indeks), mille alusel liita
- Veerud peavad olema sama andmetüübiga
- Soovitavalt unikaalsed väärtused liitmisveerus vähemalt ühes tabelis

Kõige levinum meetod on pd.merge():

In [ ]:
# Uue veeru loomine
df['income_tax'] = df['income'] * 0.24  # Näiteks tulumaksu arvutamine
df['is_adult'] = df['age'] >= 18 # tekitab boolean veeru: True/False
print(df)

# Tabelite liitmine (joinimine)
df_extra = pd.DataFrame({
    'name': ['Anna', 'Jaan', 'Mari'],
    'hobby': ['jooga', 'matkamine', 'lugemine']
})
df_joined = pd.merge(df, df_extra, on='name', how='left')
print(df_joined)

`pd.concat()` ühendab tabeleid kas ridade (`axis=0`) või veergude (`axis=1`) kaupa. Kasuta concat'i, kui soovid lihtsalt tabeleid järjestada või veerge kokku panna, mitte liita ühise veeru alusel nagu merge/join.

In [ ]:
# Tabelite ühendamine (concat) ridade kaupa
df_part1 = df.iloc[:3]
df_part2 = df.iloc[3:]
df_concat = pd.concat([df_part1, df_part2], axis=0)
print(df_concat)

# Tabelite ühendamine (concat) veergude kaupa
df_cols1 = df[['name', 'age']]
df_cols2 = df[['city', 'income']]
df_concat_cols = pd.concat([df_cols1, df_cols2], axis=1)
print(df_concat_cols)

## Andmete inspekteerimine
Enne analüüsi on oluline oma andmeid tundma õppida.

In [ ]:
# Esimeste ja viimaste ridade vaatamine
print(df.head())
print(df.tail())

`info` on üks kasulikumaid meetodeid, mis annab ülevaate veergude andmetüüpide ja puuduvatest väärtustest. See aitab mõista, milliseid andmeid on ja kas on vaja andmeid puhastada või teisendada.

In [ ]:
# Veergude info ja puuduvate väärtuste kontroll
df.info()

`describe` annab statistilise kokkuvõtte arvuliste veergude kohta (nt keskmine, mediaan, min, max, kvartilid). See aitab mõista andmete jaotust ja leida võimalikke anomaaliaid. Kokkuvõte arvutatakse ainult arvuliste veergude kohta.

In [ ]:
# Statistiline kokkuvõte
df.describe()

## Veergude ja ridade valik ning filtreerimine
Veergude valimiseks kasuta `[[]]` notatsiooni: df[['name', 'age']] valib veerud 'name' ja 'age'. Ridade ja veergude täpsemaks valikuks kasuta `.loc[]` meetodit: df.loc[<indeks>, <veerg>].

- `[[]]` notatsioon valib veerge nime järgi, nt df[['name', 'age']].
- `.loc[]` võimaldab valida ridasid ja veerge indeksi ja veeru nime järgi, nt df.loc[0, 'name'] või df.loc[df['age'] > 18, ['name', 'age']].

Filtreerimiseks kasuta tingimusi, nt df[df['age'] > 18]. Tingimusi saab kombineerida (&, |).

### Iteratiivne filtreerimine pipe meetodiga
Kui soovid järjestikku mitut filtrit rakendada, kasuta pipe meetodit, mis muudab koodi loetavamaks.

In [ ]:
# Veeru valimine [[]] notatsiooniga
alamvalik = df[['name', 'age']]
print(alamvalik)

# Ridade ja veergude valik .loc abil
valik = df.loc[df['age'] > 18, ['name', 'city']]
print(valik)

# Iteratiivne filtreerimine pipe meetodiga
df_filtered = (
    df.pipe(lambda d: d[d['age'] > 18])
      .pipe(lambda d: d[d['income'] > 1500])
      .pipe(lambda d: d[d['city'] == 'Tartu'])
)
print(df_filtered)

## Puuduvate andmete käsitlemine
Puuduvad andmed on tavalised ja nendega tuleb teadlikult ümber käia. Puuduvad väärtused võivad tekkida andmete kogumisel, vigade tõttu või seetõttu, et osa infot pole lihtsalt olemas.

**Levinumad praktikad puuduvate väärtuste käsitlemisel:**
- Puuduvate ridade eemaldamine (df.dropna()) – kasuta, kui puuduvus on juhuslik ja andmeid jääb piisavalt alles.
- Puuduvate väärtuste täitmine (df.fillna()) – kasuta, kui täitmine ei moonuta analüüsi (nt keskmise, mediaani, nulliga).
- Puuduvuse analüüs – vaata, kas puuduvus on juhuslik või süsteemne (nt ainult teatud grupis).
- Märgi puuduvad väärtused eraldi tunnusega, kui see on analüüsi jaoks oluline.

**Mida silmas pidada:**
- Kas puuduvus mõjutab analüüsi tulemusi?
- Kas täitmine või eemaldamine moonutab andmete jaotust?
- Kas eemaldamine vähendab oluliselt andmete hulka?
- Kas puuduvus on seotud mingi tunnusega (nt vanusegrupp, linn)?

Alati analüüsi puuduvate väärtuste mustrit enne otsuse tegemist!

In [ ]:
# Puuduvate väärtuste arv veergude kaupa
print(df.isnull().sum())
# Ridade eemaldamine, kus on puuduvad andmed
df_puhas = df.dropna()
# Puuduvate väärtuste täitmine
df_taidetud = df.fillna(0)  # Või df.fillna(df.mean()) arvuliste veergude jaoks

## Grupeerimine ja agregatsioon
Grupeerimine võimaldab andmeid jagada kategooriate kaupa ja rakendada igale grupile agregatsioonifunktsioone (nt keskmine, summa, min, max).

Agregatsioonifunktsioonid antakse Pandasele tavaliselt jutumärkides (nt 'mean'), sest need on Pandase sisseehitatud funktsioonide nimed. Võid kasutada ka Python funktsioone (nt np.mean, lambda vms).

**Levinumad agregatsioonifunktsioonid:**
- 'mean' – keskmine
- 'sum' – summa
- 'min' – miinimum
- 'max' – maksimum
- 'count' – ridade arv
- 'median' – mediaan
- 'std' – standardhälve

Agregatsioonifunktsioone saab kasutada nii stringina kui funktsioonina:
- df.groupby('city').agg({'income': 'mean'})
- df.groupby('city').agg({'income': np.mean})

### Custom funktsiooni kasutamine agregatsioonis
Võid defineerida oma funktsiooni ja kasutada seda groupby.agg sees. Näiteks arvutada veeru väärtuste vahe (max-min):

In [ ]:
# Grupeerimine ja erinevad agregatsioonid
import numpy as np

# Standardne agregatsioon
agg_df = df.groupby('city').agg({'age': 'mean', 'income': 'sum'})
print(agg_df)

# Mitme agregatsioonifunktsiooni kasutamine
agg_multi = df.groupby('city').agg({'income': ['mean', 'min', 'max', 'std']})
print(agg_multi)

# Custom funktsioon: vahe (max-min)
def vahe(x):
    return x.max() - x.min()
agg_custom = df.groupby('city').agg({'income': vahe})
print(agg_custom)

`groupby` võimaldab igat gruppi eraldi töödelda. Saad iga grupi nime ja selle grupi alam-DataFrame'i, mida saab analüüsida, salvestada või visualiseerida.

In [ ]:
# Prindi iga grupi alam-DataFrame
for city, group_df in df.groupby('city'):
    print(f'Grupi nimi: {city}')
    print(group_df)
    print('---')

## Andmete salvestamine faili
Töödeldud andmed saab salvestada erinevatesse formaatidesse, et neid hiljem kasutada või jagada.

In [ ]:
# Salvestamine CSV-faili
df.to_csv('output.csv', index=False)
# Salvestamine Excel-faili
df.to_excel('output.xlsx', index=False)
# Salvestamine Parquet-faili
df.to_parquet('output.parquet')

# Andmete lugemine failist
Pandas võimaldab lugeda andmeid erinevatest failiformaatidest. Allpool on näited, kuidas lugeda CSV, Excel ja Parquet faile.

In [ ]:
# CSV-faili lugemine
df_csv = pd.read_csv('andmed.csv')
# Excel-faili lugemine
df_excel = pd.read_excel('andmed.xlsx')
# Parquet-faili lugemine
df_parquet = pd.read_parquet('andmed.parquet')

## Andmete lugemine ja salvestamine
Andmeid saab Pandases lugeda ja salvestada erinevatesse failiformaatidesse. Mõned levinumad näited:
- **CSV**: Universaalne, lihtne tekstiformaat, sobib hästi andmete vahetamiseks erinevate programmide vahel. Ei säilita andmetüüpe ega vormingut.
- **Excel (XLSX)**: Mugav kontoritööks, toetab mitut lehte ja vormingut, kuid pole nii tõhus suurte andmete jaoks.
- **Parquet**: Tõhus, kiire ja kompaktne binaarformaat, sobib suurte andmete salvestamiseks ja töötlemiseks. Säilitab andmetüübid.

CSV on kõige universaalsem ja sobib andmete jagamiseks erinevate platvormide vahel. Parquet on eelistatud suurte andmete ja analüüsi jaoks, Excel sobib kontoritööks ja väiksemate andmete puhul.

In [ ]:
# Andmete salvestamine erinevatesse formaatidesse
df.to_csv('output.csv', index=False)
# df.to_excel('output.xlsx', index=False) # vajab openpyxl paketti, mida ei pruugi vaikimisi olla installitud
df.to_parquet('output.parquet')

In [21]:
# Andmete lugemine erinevatest failidest
df_csv = pd.read_csv('output.csv')
# df_excel = pd.read_excel('output.xlsx')
df_parquet = pd.read_parquet('output.parquet')